In [1]:
import numpy as np
import pandas as pd
import os
import itertools
import torch
import torch.nn as nn
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data
from torch_geometric.data import Dataset
from torch_geometric.loader import DataLoader
from sklearn.model_selection import train_test_split
from torch_geometric_temporal.signal import temporal_signal_split

In [4]:
data_df = pd.read_csv("datasetsrip - synthetic-prcp-SM-flood-data-v1.csv.csv")
data_df.drop(['Unnamed: 0'], axis=1, inplace=True)
data_df

,Prcp1,SM1,Prcp2,SM2,Prcp3,SM3,flood1,flood2,flood3
0,1.539606,0.590441,2.511432,0.474052,4.585283,0.484656,0,0,0
1,2.274125,1.249220,7.461011,2.159259,17.690303,3.474562,1,1,1
2,1.000114,0.106579,1.842560,0.010000,3.395878,0.010000,0,0,0
3,1.360013,0.429366,2.085287,0.212607,3.610686,0.137334,0,0,0
4,1.158710,0.248820,1.282190,0.010000,1.592812,0.010000,0,0,0
...,...,...,...,...,...,...,...,...,...
360,1.357030,0.426691,2.020841,0.197245,3.431358,0.103488,0,0,0
361,1.154324,0.244887,3.097722,0.201381,6.761527,0.473908,0,0,1
362,3.315756,2.183443,2.927501,2.330118,3.422077,2.060617,1,1,0
363,1.779925,0.805979,2.532323,0.718383,4.327215,0.680561,0,0,0


In [5]:
def test_1layer_RNN_2layer_GCN(df, window = 3):
    df_list = [df[i:i+window] for i in range(0,df.shape[0]-window+1,1)]

    df_dict = {}
    df_dict['x'] = [df_list[i].iloc[:,0:6] for i in range(len(df_list))]
    df_dict['y'] = [df_list[i].iloc[-1,-3:] for i in range(len(df_list))]

    
    def create_node_features(df_x):
        Prcp1 = df_x['Prcp1'].values
        Prcp2 = df_x['Prcp2'].values
        Prcp3 = df_x['Prcp3'].values
        SM1 = df_x['SM1'].values
        SM2 = df_x['SM2'].values
        SM3 = df_x['SM3'].values
        node_1 = np.hstack((Prcp1, SM1))
        node_2 = np.hstack((Prcp2, SM2))
        node_3 = np.hstack((Prcp3, SM3))
        node_features = np.vstack((node_1, node_2, node_3))
        return node_features

    data_list = []
    for i in range(len(df_dict['x'])):
        data_list.append(Data(
            x=torch.tensor(create_node_features(df_dict['x'][i]), dtype=torch.float), 
            y=torch.tensor(df_dict['y'][i].values, dtype=torch.float),
            edge_index=torch.tensor([[0,2], [1,1]], dtype=torch.long) 
            ))
        
    class RNN_GCN(nn.Module):
        def __init__(self, in_channels, out_channels):
            super(RNN_GCN, self).__init__()
            self.rnn = nn.RNN(input_size=in_channels, hidden_size=16, num_layers=1, batch_first=True)
            
            self.conv2 = GCNConv(16, out_channels)

        def forward(self, x, edge_index):
            x, h = self.rnn(x)
            x = torch.relu(x)
            x = self.conv2(x, edge_index)
            return torch.sigmoid(x)

        
        
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = RNN_GCN(in_channels=2*window, out_channels=1).to(device)
    criterion = nn.BCELoss().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

    train_ratio = 0.75  
    train_size = int(train_ratio * len(data_list))
    train_data = data_list[:train_size]
    test_data = data_list[train_size:]

    
    
    
    train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_data, batch_size=32)
    
    
    

    num_epochs = 10
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        for data in train_loader:
            data = data.to(device)
            optimizer.zero_grad()
            out = model(data.x, data.edge_index).squeeze()
            loss = criterion(out, data.y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item() * data.num_graphs
        
        print(f'Epoch {epoch+1}/{num_epochs}: Loss = {total_loss / len(train_loader.dataset):.4f}')

    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for data in test_loader:
            data = data.to(device)
            out = model(data.x, data.edge_index).squeeze()
            predicted_labels = (out > 0.5).long()
            correct += (predicted_labels == data.y).sum().item()
            total += data.y.size(0)
        
        accuracy = correct / total
        print(f'Test Accuracy: {accuracy:.4f}')

    del model
    torch.cuda.empty_cache()

test_1layer_RNN_2layer_GCN(data_df)

Epoch 1/10: Loss = 0.5859
Epoch 2/10: Loss = 0.4774
Epoch 3/10: Loss = 0.4136
Epoch 4/10: Loss = 0.3423
Epoch 5/10: Loss = 0.2900
Epoch 6/10: Loss = 0.2511
Epoch 7/10: Loss = 0.2238
Epoch 8/10: Loss = 0.2183
Epoch 9/10: Loss = 0.2193
Epoch 10/10: Loss = 0.1871
Test Accuracy: 0.9524


In [6]:
def test_1layer_RNN_2layer_GCN(df, window = 3):
    df_list = [df[i:i+window] for i in range(0,df.shape[0]-window+1,1)]

    df_dict = {}
    df_dict['x'] = [df_list[i].iloc[:,0:6] for i in range(len(df_list))]
    df_dict['y'] = [df_list[i].iloc[-1,-3:] for i in range(len(df_list))]

    
    def create_node_features(df_x):
        Prcp1 = df_x['Prcp1'].values
        Prcp2 = df_x['Prcp2'].values
        Prcp3 = df_x['Prcp3'].values
        SM1 = df_x['SM1'].values
        SM2 = df_x['SM2'].values
        SM3 = df_x['SM3'].values
        node_1 = np.hstack((Prcp1, SM1))
        node_2 = np.hstack((Prcp2, SM2))
        node_3 = np.hstack((Prcp3, SM3))
        node_features = np.vstack((node_1, node_2, node_3))
        return node_features

    data_list = []
    for i in range(len(df_dict['x'])):
        data_list.append(Data(
            x=torch.tensor(create_node_features(df_dict['x'][i]), dtype=torch.float), 
            y=torch.tensor(df_dict['y'][i].values, dtype=torch.float),
            edge_index=torch.tensor([[0,2], [1,1]], dtype=torch.long) 
            ))
        
    class RNN_GCN(nn.Module):
        def __init__(self, in_channels, out_channels):
            super(RNN_GCN, self).__init__()
            self.rnn = nn.RNN(input_size=in_channels, hidden_size=16, num_layers=1, batch_first=True)
            
            self.conv2 = GCNConv(16, out_channels)

        def forward(self, x, edge_index):
            x, h = self.rnn(x)
            x = torch.relu(x)
            x = self.conv2(x, edge_index)
            return torch.sigmoid(x)

        
        
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = RNN_GCN(in_channels=2*window, out_channels=1).to(device)
    criterion = nn.BCELoss().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

    train_ratio = 0.75  
    train_size = int(train_ratio * len(data_list))
    train_data = data_list[:train_size]
    test_data = data_list[train_size:]

    
    
    
    train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_data, batch_size=32)
    
    
    

    num_epochs = 20
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        for data in train_loader:
            data = data.to(device)
            optimizer.zero_grad()
            out = model(data.x, data.edge_index).squeeze()
            loss = criterion(out, data.y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item() * data.num_graphs
        
        print(f'Epoch {epoch+1}/{num_epochs}: Loss = {total_loss / len(train_loader.dataset):.4f}')

    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for data in test_loader:
            data = data.to(device)
            out = model(data.x, data.edge_index).squeeze()
            predicted_labels = (out > 0.5).long()
            correct += (predicted_labels == data.y).sum().item()
            total += data.y.size(0)
        
        accuracy = correct / total
        print(f'Test Accuracy: {accuracy:.4f}')

    del model
    torch.cuda.empty_cache()

test_1layer_RNN_2layer_GCN(syn_data_df)

Epoch 1/20: Loss = 0.6490
Epoch 2/20: Loss = 0.5531
Epoch 3/20: Loss = 0.4532
Epoch 4/20: Loss = 0.3563
Epoch 5/20: Loss = 0.3001
Epoch 6/20: Loss = 0.2608
Epoch 7/20: Loss = 0.2241
Epoch 8/20: Loss = 0.2049
Epoch 9/20: Loss = 0.1810
Epoch 10/20: Loss = 0.1747
Epoch 11/20: Loss = 0.1645
Epoch 12/20: Loss = 0.1504
Epoch 13/20: Loss = 0.1515
Epoch 14/20: Loss = 0.1353
Epoch 15/20: Loss = 0.1342
Epoch 16/20: Loss = 0.1320
Epoch 17/20: Loss = 0.1207
Epoch 18/20: Loss = 0.1185
Epoch 19/20: Loss = 0.1101
Epoch 20/20: Loss = 0.1065
Test Accuracy: 0.9670


In [10]:
def test_1layer_RNN_2layer_GCN(df, window = 3):
    df_list = [df[i:i+window] for i in range(0,df.shape[0]-window+1,1)]

    df_dict = {}
    df_dict['x'] = [df_list[i].iloc[:,0:6] for i in range(len(df_list))]
    df_dict['y'] = [df_list[i].iloc[-1,-3:] for i in range(len(df_list))]

    
    def create_node_features(df_x):
        Prcp1 = df_x['Prcp1'].values
        Prcp2 = df_x['Prcp2'].values
        Prcp3 = df_x['Prcp3'].values
        SM1 = df_x['SM1'].values
        SM2 = df_x['SM2'].values
        SM3 = df_x['SM3'].values
        node_1 = np.hstack((Prcp1, SM1))
        node_2 = np.hstack((Prcp2, SM2))
        node_3 = np.hstack((Prcp3, SM3))
        node_features = np.vstack((node_1, node_2, node_3))
        return node_features

    data_list = []
    for i in range(len(df_dict['x'])):
        data_list.append(Data(
            x=torch.tensor(create_node_features(df_dict['x'][i]), dtype=torch.float), 
            y=torch.tensor(df_dict['y'][i].values, dtype=torch.float),
            edge_index=torch.tensor([[0,2], [1,1]], dtype=torch.long) 
            ))
        
    class RNN_GCN(nn.Module):
        def __init__(self, in_channels, out_channels):
            super(RNN_GCN, self).__init__()
            self.rnn = nn.RNN(input_size=in_channels, hidden_size=16, num_layers=1, batch_first=True)
            
            self.conv2 = GCNConv(16, out_channels)

        def forward(self, x, edge_index):
            x, h = self.rnn(x)
            x = torch.relu(x)
            x = self.conv2(x, edge_index)
            return torch.sigmoid(x)

        
        
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = RNN_GCN(in_channels=2*window, out_channels=1).to(device)
    criterion = nn.BCELoss().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

    train_ratio = 0.75  
    train_size = int(train_ratio * len(data_list))
    train_data = data_list[:train_size]
    test_data = data_list[train_size:]

    
    
    
    train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_data, batch_size=32)
    
    
    

    num_epochs = 50
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        for data in train_loader:
            data = data.to(device)
            optimizer.zero_grad()
            out = model(data.x, data.edge_index).squeeze()
            loss = criterion(out, data.y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item() * data.num_graphs
        
        print(f'Epoch {epoch+1}/{num_epochs}: Loss = {total_loss / len(train_loader.dataset):.4f}')

    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for data in test_loader:
            data = data.to(device)
            out = model(data.x, data.edge_index).squeeze()
            predicted_labels = (out > 0.5).long()
            correct += (predicted_labels == data.y).sum().item()
            total += data.y.size(0)
        
        accuracy = correct / total
        print(f'Test Accuracy: {accuracy:.4f}')

    del model
    torch.cuda.empty_cache()

test_1layer_RNN_2layer_GCN(syn_data_df)

Epoch 1/50: Loss = 0.5807
Epoch 2/50: Loss = 0.4503
Epoch 3/50: Loss = 0.3789
Epoch 4/50: Loss = 0.3217
Epoch 5/50: Loss = 0.2941
Epoch 6/50: Loss = 0.2707
Epoch 7/50: Loss = 0.2432
Epoch 8/50: Loss = 0.2256
Epoch 9/50: Loss = 0.2104
Epoch 10/50: Loss = 0.1984
Epoch 11/50: Loss = 0.1887
Epoch 12/50: Loss = 0.1900
Epoch 13/50: Loss = 0.1869
Epoch 14/50: Loss = 0.1742
Epoch 15/50: Loss = 0.1656
Epoch 16/50: Loss = 0.1677
Epoch 17/50: Loss = 0.1559
Epoch 18/50: Loss = 0.1573
Epoch 19/50: Loss = 0.1473
Epoch 20/50: Loss = 0.1527
Epoch 21/50: Loss = 0.1488
Epoch 22/50: Loss = 0.1498
Epoch 23/50: Loss = 0.1527
Epoch 24/50: Loss = 0.1464
Epoch 25/50: Loss = 0.1488
Epoch 26/50: Loss = 0.1419
Epoch 27/50: Loss = 0.1444
Epoch 28/50: Loss = 0.1377
Epoch 29/50: Loss = 0.1309
Epoch 30/50: Loss = 0.1301
Epoch 31/50: Loss = 0.1294
Epoch 32/50: Loss = 0.1263
Epoch 33/50: Loss = 0.1220
Epoch 34/50: Loss = 0.1262
Epoch 35/50: Loss = 0.1136
Epoch 36/50: Loss = 0.1237
Epoch 37/50: Loss = 0.1308
Epoch 38/5

In [9]:
def test_1layer_RNN_2layer_GCN(df, window = 3):
    df_list = [df[i:i+window] for i in range(0,df.shape[0]-window+1,1)]

    df_dict = {}
    df_dict['x'] = [df_list[i].iloc[:,0:6] for i in range(len(df_list))]
    df_dict['y'] = [df_list[i].iloc[-1,-3:] for i in range(len(df_list))]

    
    def create_node_features(df_x):
        Prcp1 = df_x['Prcp1'].values
        Prcp2 = df_x['Prcp2'].values
        Prcp3 = df_x['Prcp3'].values
        SM1 = df_x['SM1'].values
        SM2 = df_x['SM2'].values
        SM3 = df_x['SM3'].values
        node_1 = np.hstack((Prcp1, SM1))
        node_2 = np.hstack((Prcp2, SM2))
        node_3 = np.hstack((Prcp3, SM3))
        node_features = np.vstack((node_1, node_2, node_3))
        return node_features

    data_list = []
    for i in range(len(df_dict['x'])):
        data_list.append(Data(
            x=torch.tensor(create_node_features(df_dict['x'][i]), dtype=torch.float), 
            y=torch.tensor(df_dict['y'][i].values, dtype=torch.float),
            edge_index=torch.tensor([[0,2], [1,1]], dtype=torch.long) 
            ))
        
    class RNN_GCN(nn.Module):
        def __init__(self, in_channels, out_channels):
            super(RNN_GCN, self).__init__()
            self.rnn = nn.RNN(input_size=in_channels, hidden_size=16, num_layers=1, batch_first=True)
            
            self.conv2 = GCNConv(16, out_channels)

        def forward(self, x, edge_index):
            x, h = self.rnn(x)
            x = torch.relu(x)
            x = self.conv2(x, edge_index)
            return torch.sigmoid(x)

        
        
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = RNN_GCN(in_channels=2*window, out_channels=1).to(device)
    criterion = nn.BCELoss().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

    train_ratio = 0.75  
    train_size = int(train_ratio * len(data_list))
    train_data = data_list[:train_size]
    test_data = data_list[train_size:]

    
    
    
    train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_data, batch_size=32)
    
    
    

    num_epochs = 100
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        for data in train_loader:
            data = data.to(device)
            optimizer.zero_grad()
            out = model(data.x, data.edge_index).squeeze()
            loss = criterion(out, data.y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item() * data.num_graphs
        
        print(f'Epoch {epoch+1}/{num_epochs}: Loss = {total_loss / len(train_loader.dataset):.4f}')

    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for data in test_loader:
            data = data.to(device)
            out = model(data.x, data.edge_index).squeeze()
            predicted_labels = (out > 0.5).long()
            correct += (predicted_labels == data.y).sum().item()
            total += data.y.size(0)
        
        accuracy = correct / total
        print(f'Test Accuracy: {accuracy:.4f}')

    del model
    torch.cuda.empty_cache()

test_1layer_RNN_2layer_GCN(syn_data_df)

Epoch 1/100: Loss = 0.5588
Epoch 2/100: Loss = 0.4437
Epoch 3/100: Loss = 0.3651
Epoch 4/100: Loss = 0.3095
Epoch 5/100: Loss = 0.2792
Epoch 6/100: Loss = 0.2482
Epoch 7/100: Loss = 0.2381
Epoch 8/100: Loss = 0.2259
Epoch 9/100: Loss = 0.2020
Epoch 10/100: Loss = 0.1888
Epoch 11/100: Loss = 0.1810
Epoch 12/100: Loss = 0.1732
Epoch 13/100: Loss = 0.1710
Epoch 14/100: Loss = 0.1638
Epoch 15/100: Loss = 0.1579
Epoch 16/100: Loss = 0.1525
Epoch 17/100: Loss = 0.1440
Epoch 18/100: Loss = 0.1371
Epoch 19/100: Loss = 0.1377
Epoch 20/100: Loss = 0.1395
Epoch 21/100: Loss = 0.1270
Epoch 22/100: Loss = 0.1237
Epoch 23/100: Loss = 0.1111
Epoch 24/100: Loss = 0.1149
Epoch 25/100: Loss = 0.1115
Epoch 26/100: Loss = 0.1044
Epoch 27/100: Loss = 0.1098
Epoch 28/100: Loss = 0.1023
Epoch 29/100: Loss = 0.1076
Epoch 30/100: Loss = 0.0956
Epoch 31/100: Loss = 0.0939
Epoch 32/100: Loss = 0.0904
Epoch 33/100: Loss = 0.0894
Epoch 34/100: Loss = 0.0827
Epoch 35/100: Loss = 0.0799
Epoch 36/100: Loss = 0.0753
E